In [10]:
from pathlib import Path

from ultralytics.utils.downloads import download

dir = Path(r"C:\Users\Navdeep\datasets\VisDrone")  # dataset root dir
urls = ['https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-train.zip',
          'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-val.zip',
          'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-dev.zip',
          'https://github.com/ultralytics/yolov5/releases/download/v1.0/VisDrone2019-DET-test-challenge.zip']
download(urls, dir=dir, curl=True, threads=4)


In [16]:
def visdrone2yolo(dir):
      from PIL import Image
      from tqdm import tqdm

      def convert_box(size, box):
          # Convert VisDrone box to YOLO xywh box
          dw = 1. / size[0]
          dh = 1. / size[1]
          return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

      (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
      pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
      for f in pbar:
          img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
          lines = []
          with open(f, 'r') as file:  # read annotation.txt
              for row in [x.split(',') for x in file.read().strip().splitlines()]:
                  if row[4] == '0':  # VisDrone 'ignored regions' class 0
                      continue
                  cls = int(row[5]) - 1
                  box = convert_box(img_size, tuple(map(int, row[:4])))
                  lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                  with open(str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}'), 'w') as fl:
                      fl.writelines(lines)  # write label.txt


In [17]:
# Convert
import os
for d in 'VisDrone2019-DET-train', 'VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev':
    visdrone2yolo(dir / d)  # convert VisDrone annotations to YOLO labels

Converting C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-train: 6471it [02:39, 40.56it/s] 
Converting C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-val: 548it [00:17, 31.33it/s]
Converting C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-test-dev: 1610it [00:35, 45.68it/s]


In [1]:
import torch

# Check the number of available GPUs
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    # Set the active GPU to the first available GPU
    torch.cuda.set_device(0)
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPUs available on this system.")


Using GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO(r'C:\Users\Navdeep\runs\detect\train13\weights\last.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(resume = True)

New https://pypi.org/project/ultralytics/8.1.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.0 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\Navdeep\runs\detect\train13\weights\last.pt, data=VisDrone.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=0, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, r

train: Scanning C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-train\labels.cache... 6471 images, 0 backgrounds, 0

train: WARNING ⚠️ C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-train\images\0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-train\images\0000140_00118_d_0000002.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-train\images\9999945_00000_d_0000114.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-train\images\9999987_00000_d_0000049.jpg: 1 duplicate labels removed



val: Scanning C:\Users\Navdeep\datasets\VisDrone\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corr


Plotting labels to runs\detect\train13\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training from C:\Users\Navdeep\runs\detect\train13\weights\last.pt from epoch 87 to 100 total epochs
100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        548      38759      0.436      0.334      0.335      0.195

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      6.68G       1.27     0.8576     0.8983        307        640: 100%|██████████| 405/405 [08:32<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:15


                   all        548      38759      0.433      0.335      0.335      0.195

14 epochs completed in 1.537 hours.
Optimizer stripped from runs\detect\train13\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train13\weights\best.pt, 6.2MB

Validating runs\detect\train13\weights\best.pt...
Ultralytics YOLOv8.1.0 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:23


                   all        548      38759      0.442      0.336      0.338      0.197
            pedestrian        548       8844       0.45      0.348       0.35       0.15
                people        548       5125        0.5      0.236       0.28        0.1
               bicycle        548       1287      0.195      0.112     0.0808     0.0314
                   car        548      14064       0.64       0.75      0.758       0.52
                   van        548       1975      0.468      0.381      0.386      0.266
                 truck        548        750      0.468      0.299       0.32      0.212
              tricycle        548       1045      0.403      0.251      0.229      0.124
       awning-tricycle        548        532      0.276      0.154      0.127     0.0788
                   bus        548        251      0.558      0.462      0.486      0.339
                 motor        548       4886      0.461      0.369      0.368      0.152
Speed: 0.4ms preproce

In [3]:
!pip install blosc2~=2.0.0

     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
      --------------------------------------- 0.0/1.9 MB 435.7 kB/s eta 0:00:05
      --------------------------------------- 0.0/1.9 MB 393.8 kB/s eta 0:00:05
      --------------------------------------- 0.0/1.9 MB 393.8 kB/s eta 0:00:05
      --------------------------------------- 0.0/1.9 MB 393.8 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.9 MB 655.8 kB/s eta 0:00:03
     ----- ---------------------------------- 0.3/1.9 MB 1.1 MB/s eta 0:00:02
     ---------- ----------------------------- 0.5/1.9 MB 1.7 MB/s eta 0:00:01
     ------------ --------------------------- 0.6/1.9 MB 1.9 MB/s eta 0:00:01
     ------------------ --------------------- 0.9/1.9 MB 2.3 MB/s eta 0:00:01
     -------------------- ------------------- 1.0/1.9 MB 2.5 MB/s eta 0:00:01
     -------------------- ------------------- 1.0/1.9 MB 2.5 MB/s et

In [17]:
!pip install boxmot

In [9]:
!pip list --path C:\Users\Navdeep\anaconda3\Lib\site-packages

Package                       Version
----------------------------- --------------------
absl-py                       2.0.0
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs

In [7]:
import boxmot

ModuleNotFoundError: No module named 'boxmot'

In [13]:
!pip show blosc2

Name: blosc2
Version: 2.0.0
Summary: Python wrapper for the C-Blosc2 library.
Home-page: https://github.com/Blosc/python-blosc2
Author: Blosc Development Team
Author-email: blosc@blosc.org
License: https://opensource.org/licenses/BSD-3-Clause
Location: C:\Users\Navdeep\anaconda3\Lib\site-packages
Requires: msgpack
Required-by: tables


In [11]:
!pip show filterpy

Name: filterpy
Version: 1.4.5
Summary: Kalman filtering and optimal estimation library
Home-page: https://github.com/rlabbe/filterpy
Author: Roger Labbe
Author-email: rlabbejr@gmail.com
License: MIT
Location: C:\Users\Navdeep\AppData\Roaming\Python\Python311\site-packages
Requires: matplotlib, numpy, scipy
Required-by: boxmot


In [15]:
!pip uninstall boxmot

^C


In [18]:
!pip show boxmot

Name: boxmot
Version: 10.0.51
Summary: SOTA tracking methods for detection, segmentation and pose estimation models.
Home-page: https://github.com/mikel-brostrom/yolo_tracking
Author: Mikel Broström
Author-email: yolov5.deepsort.pytorch@gmail.com
License: AGPL-3.0
Location: C:\Users\Navdeep\AppData\Roaming\Python\Python311\site-packages
Requires: filterpy, ftfy, gdown, GitPython, lapx, loguru, numpy, opencv-python, pandas, pre-commit, PyYAML, regex, scikit-learn, tensorboard, torch, torchvision, yacs
Required-by: 


In [19]:
import sys
sys.path.append(r'C:\Users\Navdeep\AppData\Roaming\Python\Python311\site-packages')

In [21]:
import cv2
import numpy as np
from pathlib import Path

from boxmot import DeepOCSORT


tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device='cuda:0',
    fp16=False,
)

Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: C:\Users\Navdeep\osnet_x0_25_msmt17.pt
100%|█████████████████████████████████████████████████████████████████████████████| 3.06M/3.06M [00:01<00:00, 3.06MB/s]
2024-01-17 13:41:21.096 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-01-17 13:41:21.098 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


In [4]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO(r"C:\Users\Navdeep\runs\detect\train13\weights\best.pt")
model.to('cpu')
# Run inference on 'bus.jpg' with arguments
model.predict(r"C:\Part_dimensions_project\frames\frame_0002.jpg", save=True, imgsz=320, conf=0.5)


image 1/1 C:\Part_dimensions_project\frames\frame_0002.jpg: 192x320 2 cars, 44.2ms
Speed: 1.0ms preprocess, 44.2ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 320)
Results saved to runs\detect\predict4


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'pedestrian', 1: 'people', 2: 'bicycle', 3: 'car', 4: 'van', 5: 'truck', 6: 'tricycle', 7: 'awning-tricycle', 8: 'bus', 9: 'motor'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]